In [1]:
#preprocess ml
#train
#test

In [2]:
import pandas as pd
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,accuracy_score,roc_auc_score
from sklearn.ensemble import GradientBoostingRegressor,ExtraTreesClassifier #随机森林分类模型
import numpy as np
import os

In [ ]:
symbol_s = ['SC', 'PG', 'AU', 'AG', 'CU', 'PB', 'NI', 'BC', 'A', 'B', 'M', 'JM', 'J', 'C', 'CS', 'MA', 'PP', 'CU', 'RU', 'Y', 'P', 'AU', 'AG', 'CU', 'AL', 'ZN', 'PB', 'NI', 'SN', 'RB', 'HC', 'BU', 'RU', 'FU', 'SP', 'WR', 'SS', 'SA', 'UR', 'SR', 'CF', 'TA', 'MA', 'FG', 'RS', 'OI', 'AU', 'AG', 'JR', 'SF', 'SM', 'AP', 'CJ', 'CY', 'RR', 'BB', 'FB', 'EB', 'V', 'L', 'PP', 'CS', 'JD', 'BU', 'HC','IC','IH','IF']
for symbol in symbol_s:
    print(symbol)
    m = 10
    print(m/60,m/(24*60),'day')
    test_size = 0.4
    train_size = 1 - test_size
    df = []
    csv_filename = f"./{symbol}_1min.csv"  # Replace with the actual filename
    df = pd.read_csv(csv_filename)
    df.loc[:, 'open':'hold'] = df.loc[:, 'open':'hold'].apply(pd.to_numeric, errors='coerce')
    df = df.dropna()
    #df.loc[:,'open':'close'] = df.loc[:,'open': 'close'].astype(float)
    # 处理时间间隔
    df['datetime'] = pd.to_datetime(df['datetime'])
    df["(t+1)-(t)"] = np.nan
    time_threshold_window = pd.DateOffset(minutes=m)
     ######-----------计算label
    for index, row in df.iterrows():
        target_time = row['datetime'] + time_threshold_window
        result_row = df.loc[df['datetime'] == target_time]
        if not result_row.empty:
            #print(result_row['close'].iloc[0])
            df.loc[index, "(t+1)-(t)"] = result_row['close'].iloc[0] - row['close']
    df = df.dropna()
    #print(df)
    df['label'] = 0
    # 构建对应表
    df = df.dropna()
    ##去除异常
    df.drop(df[abs(df["(t+1)-(t)"]) > 10000].index, inplace=True)
    ##
    ################
    sigma = np.std(df["(t+1)-(t)"][: int(train_size*len(df))])
    mean = np.mean(df["(t+1)-(t)"][: int(train_size*len(df))])
    df['label'] = df["(t+1)-(t)"].apply(lambda x: 1 if x > mean + 3 * sigma else (-1 if x < mean - 3 * sigma else 0))
    df = df.dropna()
    #####================
    os.makedirs(f'{symbol}', exist_ok=True)
    df.to_csv(f'./{symbol}/{symbol}{m}_ans.csv',mode='w')
    #time_threshold


    class RF_predict:
        def __init__(self, _df):
            self.tsData = _df
            self.test_data = []
        def make_predict(self, size ,test_data):
            #node = test_size
            node = int(len(self.tsData) *size)
            #
            #print(self.tsData)
            # 构建数据集
            train_data = self.tsData[: len(self.tsData) - int(node)]
            self.test_data = self.tsData[len(self.tsData) - node : ]
            train_X = train_data.loc[:, 'open': 'hold'].values
            train_y = train_data['label'].values

            test_X = self.test_data.loc[:, 'open': 'hold'].values
            test_y = self.test_data['label'].values
    
            # 进行预测
            clf = ExtraTreesClassifier()
            clf.fit(train_X, train_y)
            print(accuracy_score(train_y, clf.predict(train_X)))
            print(accuracy_score(test_y, clf.predict(test_X)),'\n')
            #print(roc_auc_score(test_y,clf.predict(test_X))) # 召回率
            print()

    # a = RF_predict(df)
    # a.make_predict(test_size,[])
    # a.test_data

SC
0.16666666666666666 0.006944444444444444 day
PG
0.16666666666666666 0.006944444444444444 day
AU
0.16666666666666666 0.006944444444444444 day
AG
0.16666666666666666 0.006944444444444444 day


# 

In [ ]:
symbol_s = ['SC', 'PG', 'AU', 'AG', 'CU', 'PB', 'NI', 'BC', 'A', 'B', 'M', 'JM', 'J', 'C', 'CS', 'MA', 'PP', 'CU', 'RU', 'Y', 'P', 'AU', 'AG', 'CU', 'AL', 'ZN', 'PB', 'NI', 'SN', 'RB', 'HC', 'BU', 'RU', 'FU', 'SP', 'WR', 'SS', 'SA', 'UR', 'SR', 'CF', 'TA', 'MA', 'FG', 'RS', 'OI', 'AU', 'AG', 'JR', 'SF', 'SM', 'AP', 'CJ', 'CY', 'RR', 'BB', 'FB', 'EB', 'V', 'L', 'PP', 'CS', 'JD', 'BU', 'HC','IC','IH','IF']

sigma_map = {}
for symbol_ in symbol_s:
    for m in range(10,20,10):
        print(m/60,m/(24*60))
        df = []
        csv_filename = f'./{symbol_}/{symbol_}{m}_ans.csv'  # Replace with the actual filename
        df = pd.read_csv(csv_filename)
        sigma = np.std(df["(t+1)-(t)"])
        mean = np.mean(df["(t+1)-(t)"])

        sigma_map.append(symbol_s,sigma)
        print(sigma)
        print(mean)

print(sigma_map)

In [ ]:
['SC', 'PG', 'AU', 'AG', 'CU', 'PB', 'NI', 'BC', 'A', 'B', 'M', 'JM', 'J', 'C', 'CS', 'MA', 'PP', 'CU', 'RU', 'Y', 'P', 'AU', 'AG', 'CU', 'AL', 'ZN', 'PB', 'NI', 'SN', 'RB', 'HC', 'BU', 'RU', 'FU', 'SP', 'WR', 'SS', 'SA', 'UR', 'SR', 'CF', 'TA', 'MA', 'FG', 'RS', 'OI', 'AU', 'AG', 'JR', 'SF', 'SM', 'AP', 'CJ', 'CY', 'RR', 'BB', 'FB', 'EB', 'V', 'L', 'PP', 'CS', 'JD', 'BU', 'HC','IC','IH','IF
']